In [103]:
# ==========================================
# Cell 1: Load requirements.json (with stakeholders)
# ==========================================
import json
import pathlib
import pandas as pd

def find_repo_root(start=pathlib.Path.cwd()):
    for cand in [start] + list(start.parents):
        if (cand / 'build.gradle').exists() or (cand / '.git').exists():
            return cand
    return pathlib.Path.cwd()

repo_root = find_repo_root()
req_path = repo_root / 'build' / 'results' / 'mission-control' / 'requirements.json'
if not req_path.exists():
    raise FileNotFoundError(f"Requirements file not found at {req_path} (repo_root={repo_root})")

with req_path.open() as f:
    data = json.load(f)

bindings = data.get('results', {}).get('bindings', [])
rows = []

def short_from_uri(u):
    if not u:
        return ''
    if '#' in u:
        return u.split('#')[-1]
    return u.rstrip('/').split('/')[-1]

for b in bindings:
    req_uri = b.get('requirementURI', {}).get('value')
    desc = b.get('description', {}).get('value')
    expr = b.get('expression', {}).get('value')
    stakeholder_uri = b.get('stakeholderURI', {}).get('value')

    name = short_from_uri(req_uri)
    stakeholder = short_from_uri(stakeholder_uri)

    rows.append({
        'Name': name,
        'Description': desc,
        'Expression': expr,
        'Stakeholder': stakeholder
    })

df = pd.DataFrame(rows)
df

,Name,Description,Expression,Stakeholder
0,R1,Real-Time Map,The system shall display real-time fire and dr...,Operator
1,R2,Critical Alerts,The system shall show real-time critical alert...,SafetyOfficer
2,R3,AI Warden Interface,"The system shall enable direct, smooth communi...",Operator
3,R4,Adaptive User Experience,The system shall remain functional under field...,FireFighter
4,R5,Historical Views,The system shall support views of historical f...,Operator
5,R6,Interoperability with Fire Cloud,The system shall ingest and interpret data str...,SystemEngineer
6,R7,Decision Approval Mechanism,The system shall allow human approval for AI F...,CommandCenter


In [104]:
# ==========================================
# Cell 2: Build mission-capability-requirement graph
# ==========================================
import networkx as nx
import plotly.graph_objects as go

cap_path = repo_root / 'build' / 'results' / 'mission-control' / 'missions_capabilities.json'
if not cap_path.exists():
    raise FileNotFoundError(f"Capabilities file not found at {cap_path} (repo_root={repo_root})")

with cap_path.open() as f:
    data = json.load(f)

bindings = data.get('results', {}).get('bindings', [])

nodes = {}
edges = set()

for b in bindings:
    req_uri = b.get('requirementURI', {}).get('value')
    req_desc = b.get('requirementDescription', {}).get('value')
    cap_uri = b.get('capabilityURI', {}).get('value')
    cap_desc = b.get('capabilityDescription', {}).get('value')
    sub_uri = b.get('subCapabilityURI', {}).get('value')
    sub_desc = b.get('subCapabilityDescription', {}).get('value')

    req_name = short_from_uri(req_uri)
    cap_name = short_from_uri(cap_uri)
    sub_name = short_from_uri(sub_uri) if sub_uri else None

    if req_name:
        if req_name not in nodes:
            nodes[req_name] = req_desc
    if cap_name:
        if cap_name not in nodes:
            nodes[cap_name] = cap_desc
    if sub_name:
        if sub_name not in nodes:
            nodes[sub_name] = sub_desc

    # requirement → capability
    if req_name and cap_name:
        edges.add((req_name, cap_name))
    # capability → subcapability
    if cap_name and sub_name:
        edges.add((cap_name, sub_name))

G = nx.Graph()
for n, d in nodes.items():
    G.add_node(n, description=d)
for a, b in edges:
    G.add_edge(a, b)

pos = nx.spring_layout(G, seed=42)

# --- Build Plotly traces ---
edge_x, edge_y = [], []
for u, v in G.edges():
    x0, y0 = pos[u]
    x1, y1 = pos[v]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y, mode='lines',
    line=dict(width=1, color='#888'),
    hoverinfo='none'
)

node_x, node_y, hover_text, labels, node_colors = [], [], [], [], []
for n in G.nodes():
    x, y = pos[n]
    node_x.append(x)
    node_y.append(y)
    desc = G.nodes[n].get('description') or ''
    hover_text.append(f"{n}<br>{desc}")
    labels.append(n)
    
    # Color-code by prefix: R=Requirements (coral), C=Capabilities (lightblue), M=Missions (lightgreen)
    if n.startswith('R'):
        node_colors.append('LightCoral')
    elif n.startswith('C'):
        node_colors.append('LightSkyBlue')
    elif n.startswith('M'):
        node_colors.append('LightGreen')
    else:
        node_colors.append('LightGray')

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    text=labels,
    textposition='top center',
    hovertext=hover_text,
    hoverinfo='text',
    marker=dict(size=18, color=node_colors, line=dict(width=1, color='DarkSlateGrey'))
)

fig_missions = go.Figure(
    data=[edge_trace, node_trace],
    layout=go.Layout(
        title='Requirement–Capability–SubCapability Graph',
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40)
    )
)

fig_missions.show()


In [105]:
# ==========================================
# Cell 3: Capability → Entity Graph
# ==========================================
ent_path = repo_root / 'build' / 'results' / 'mission-control' / 'entities.json'
if not ent_path.exists():
    raise FileNotFoundError(f"Entities file not found at {ent_path} (repo_root={repo_root})")

with ent_path.open() as f:
    data = json.load(f)

bindings = data.get('results', {}).get('bindings', [])
nodes = {}
edges = set()

for b in bindings:
    cap_uri = b.get('capabilityURI', {}).get('value')
    cap_desc = b.get('capabilityDescription', {}).get('value')
    ent_uri = b.get('entityURI', {}).get('value')
    ent_desc = b.get('entityDescription', {}).get('value')

    cap_name = short_from_uri(cap_uri)
    ent_name = short_from_uri(ent_uri)

    if cap_name:
        if cap_name not in nodes:
            nodes[cap_name] = cap_desc
    if ent_name:
        if ent_name not in nodes:
            nodes[ent_name] = ent_desc
    if cap_name and ent_name:
        edges.add((cap_name, ent_name))

G_entities = nx.Graph()
for n, d in nodes.items():
    G_entities.add_node(n, description=d)
for a, b in edges:
    G_entities.add_edge(a, b)

pos = nx.spring_layout(G_entities, seed=42)

# --- Build edge trace ---
edge_x, edge_y = [], []
for u, v in G_entities.edges():
    x0, y0 = pos[u]
    x1, y1 = pos[v]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y, mode='lines',
    line=dict(width=1, color='#888'),
    hoverinfo='none'
)

# --- Build node trace ---
node_x, node_y, hover_text, labels, node_colors = [], [], [], [], []
for n in G_entities.nodes():
    x, y = pos[n]
    node_x.append(x)
    node_y.append(y)
    desc = G_entities.nodes[n].get('description') or ''
    hover_text.append(f"{n}<br>{desc}")
    labels.append(n)
    
    # Color-code: capabilities (C prefix) = lightblue, entities (others) = lightgreen
    if n.startswith('C') and n[1:].isdigit():  # e.g., C1, C2, C3...
        node_colors.append('LightSkyBlue')
    else:  # entity names like SafetyOfficer, FireDrone, etc.
        node_colors.append('LightGreen')

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    text=labels,
    textposition='top center',
    hovertext=hover_text,
    hoverinfo='text',
    marker=dict(size=18, color=node_colors, line=dict(width=1, color='DarkSlateGrey'))
)

fig_entities = go.Figure(
    data=[edge_trace, node_trace],
    layout=go.Layout(
        title='Capability–Entity Graph',
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40)
    )
)

fig_entities.show()


In [106]:
# ==========================================
# Cell 4: Activities → Data Flow Graph
# ==========================================
act_path = repo_root / 'build' / 'results' / 'mission-control' / 'activities.json'
if not act_path.exists():
    raise FileNotFoundError(f"Activities file not found at {act_path} (repo_root={repo_root})")

with act_path.open() as f:
    data = json.load(f)

bindings = data.get('results', {}).get('bindings', [])
nodes = {}  # activity_name -> description
edges = {}  # (a1, a2) -> (data_uri, datadesc)

for b in bindings:
    a1_uri = b.get('a1', {}).get('value')
    a1_desc = b.get('desc1', {}).get('value')
    a2_uri = b.get('a2', {}).get('value')
    a2_desc = b.get('desc2', {}).get('value')
    data_uri = b.get('d', {}).get('value')
    datadesc = b.get('datadesc', {}).get('value')

    a1_name = short_from_uri(a1_uri)
    a2_name = short_from_uri(a2_uri)
    data_name = short_from_uri(data_uri)

    if a1_name:
        if a1_name not in nodes:
            nodes[a1_name] = a1_desc
    if a2_name:
        if a2_name not in nodes:
            nodes[a2_name] = a2_desc
    
    # Store edge with data URI and description
    if a1_name and a2_name:
        edges[(a1_name, a2_name)] = (data_name, datadesc)

G_activities = nx.DiGraph()  # Directed graph for activities
for n, d in nodes.items():
    G_activities.add_node(n, description=d)
for (a1, a2), (data_name, dd) in edges.items():
    G_activities.add_edge(a1, a2, data_name=data_name, datadesc=dd)

pos = nx.spring_layout(G_activities, seed=42)

# --- Build edge traces (lines only, no hover) ---
edge_x, edge_y = [], []
for u, v in G_activities.edges():
    x0, y0 = pos[u]
    x1, y1 = pos[v]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    mode='lines',
    line=dict(width=2, color='#888'),
    hoverinfo='none',
    showlegend=False
)

# --- Build edge label trace (at midpoint of each edge) ---
edge_label_x, edge_label_y, edge_label_text, edge_label_hover = [], [], [], []
for u, v in G_activities.edges():
    x0, y0 = pos[u]
    x1, y1 = pos[v]
    # Midpoint of edge
    mid_x = (x0 + x1) / 2
    mid_y = (y0 + y1) / 2
    
    data_name = G_activities.edges[u, v].get('data_name', '')
    datadesc = G_activities.edges[u, v].get('datadesc', '')
    
    edge_label_x.append(mid_x)
    edge_label_y.append(mid_y)
    edge_label_text.append(data_name)
    edge_label_hover.append(f"{data_name}<br>{datadesc}")

edge_label_trace = go.Scatter(
    x=edge_label_x, y=edge_label_y,
    mode='markers+text',
    text=edge_label_text,
    textposition='middle center',
    textfont=dict(size=10, color='#555'),
    hovertext=edge_label_hover,
    hoverinfo='text',
    marker=dict(size=8, color='#FFD700', opacity=0.7, line=dict(width=0.5, color='#888')),
    showlegend=False
)

# --- Build node trace ---
node_x, node_y, hover_text, labels, node_colors = [], [], [], [], []
for n in G_activities.nodes():
    x, y = pos[n]
    node_x.append(x)
    node_y.append(y)
    desc = G_activities.nodes[n].get('description') or ''
    hover_text.append(f"{n}<br>{desc}")
    labels.append(n)
    
    # Color-code activities (A prefix) with a distinct color
    if n.startswith('A') and n[1:].isdigit():
        node_colors.append('LightSalmon')
    else:
        node_colors.append('LightGray')

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    text=labels,
    textposition='top center',
    hovertext=hover_text,
    hoverinfo='text',
    marker=dict(size=20, color=node_colors, line=dict(width=1, color='DarkSlateGrey')),
    showlegend=False
)

fig_activities = go.Figure(
    data=[edge_trace, edge_label_trace, node_trace],
    layout=go.Layout(
        title='Activity–Data Flow Graph',
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40)
    )
)

fig_activities.show()


In [107]:
# ==========================================
# Cell 5: State Machine Graph (using Mermaid)
# ==========================================
import base64
from IPython.display import Image, display
import html as html_module

sm_path = repo_root / 'build' / 'results' / 'mission-control' / 'statemachine.json'
if not sm_path.exists():
    raise FileNotFoundError(f"State machine file not found at {sm_path} (repo_root={repo_root})")

with sm_path.open() as f:
    data = json.load(f)

bindings = data.get('results', {}).get('bindings', [])
nodes = {}  # state_name -> description
edges = {}  # (s1, s2) -> (trigger_name, triggerdesc)

for b in bindings:
    s1_uri = b.get('s1', {}).get('value')
    s1_desc = b.get('s1desc', {}).get('value')
    s2_uri = b.get('s2', {}).get('value')
    s2_desc = b.get('s2desc', {}).get('value')
    trigger_uri = b.get('trigger', {}).get('value')
    trigger_desc = b.get('triggerdesc', {}).get('value')

    s1_name = short_from_uri(s1_uri)
    s2_name = short_from_uri(s2_uri)
    trigger_name = short_from_uri(trigger_uri) if trigger_uri else None

    if s1_name:
        if s1_name not in nodes:
            nodes[s1_name] = s1_desc
    if s2_name:
        if s2_name not in nodes:
            nodes[s2_name] = s2_desc
    
    # Store edge with trigger URI and description
    if s1_name and s2_name:
        edges[(s1_name, s2_name)] = (trigger_name, trigger_desc)

# Build Mermaid state diagram
mermaid_code = "stateDiagram-v2\n"
mermaid_code += "    direction LR\n\n"

# Add transitions with trigger labels (no state descriptions in diagram - those will be tooltips)
for (s1, s2), (trigger_name, trigger_desc) in edges.items():
    if trigger_name:
        # Use trigger name as the transition label
        clean_trigger = trigger_name.replace('"', "'")
        mermaid_code += f'    {s1} --> {s2} : {clean_trigger}\n'
    else:
        mermaid_code += f'    {s1} --> {s2}\n'

# Store for HTML export (we'll need the data for tooltips)
fig_statemachine_mermaid = mermaid_code
statemachine_nodes = nodes  # Store node descriptions
statemachine_edges = edges  # Store edge/trigger descriptions

# Helper functions for Mermaid rendering via mermaid.ink
def mm_ink(graphbytes):
    """Given a bytes object holding a Mermaid-format graph, return a URL that will generate the image."""
    base64_bytes = base64.b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    return "https://mermaid.ink/img/" + base64_string

def mm_display(graphbytes):
    """Given a bytes object holding a Mermaid-format graph, display it."""
    display(Image(url=mm_ink(graphbytes)))

def mm(graph):
    """Given a string containing a Mermaid-format graph, display it."""
    graphbytes = graph.encode("utf-8")
    mm_display(graphbytes)

# Render the Mermaid diagram in Jupyter
print("State Machine Diagram (Mermaid):")
print("=" * 60)
print("Note: Hover over states/transitions in HTML export for full descriptions.")
print()
mm(mermaid_code)


State Machine Diagram (Mermaid):
Note: Hover over states/transitions in HTML export for full descriptions.



In [108]:
# ==========================================
# Cell 6: Scenarios Timeline Visualization
# ==========================================
import plotly.graph_objects as go
from collections import defaultdict
import re

scenarios_path = repo_root / 'build' / 'results' / 'mission-control' / 'scenarios.json'
if not scenarios_path.exists():
    raise FileNotFoundError(f"Scenarios file not found at {scenarios_path} (repo_root={repo_root})")

with scenarios_path.open() as f:
    data = json.load(f)

bindings = data.get('results', {}).get('bindings', [])

# Group data by scenario
scenarios_data = defaultdict(lambda: {
    'description': '',
    'lifelines': {},  # lifeline_name -> description
    'timepoints': defaultdict(lambda: defaultdict(lambda: {'descriptions': set(), 'types': set()}))  # tp_name -> lifeline_name -> {descriptions, types}
})

for b in bindings:
    scenario_uri = b.get('scenario', {}).get('value')
    scenario_desc = b.get('scenarioDesc', {}).get('value')
    lifeline_uri = b.get('lifeline', {}).get('value')
    lifeline_desc = b.get('lifelineDesc', {}).get('value')
    timepoint_uri = b.get('timepoint', {}).get('value')
    message_desc = b.get('messageDesc', {}).get('value')
    execution_desc = b.get('executionDesc', {}).get('value')
    state_frag_desc = b.get('stateFragmentDesc', {}).get('value')
    
    scenario_name = short_from_uri(scenario_uri)
    
    if scenario_name:
        scenarios_data[scenario_name]['description'] = scenario_desc or ''
        
        if lifeline_uri:
            lifeline_name = short_from_uri(lifeline_uri)
            
            # Store lifeline description
            if lifeline_name not in scenarios_data[scenario_name]['lifelines']:
                scenarios_data[scenario_name]['lifelines'][lifeline_name] = lifeline_desc or ''
            
            if timepoint_uri:
                tp_name = short_from_uri(timepoint_uri)
                
                # Track unique descriptions and their types
                event_data = scenarios_data[scenario_name]['timepoints'][tp_name][lifeline_name]
                
                # Add descriptions to set, tracking which type each belongs to
                if message_desc and message_desc not in event_data['descriptions']:
                    event_data['descriptions'].add(message_desc)
                    event_data['types'].add('message')
                
                if execution_desc and execution_desc not in event_data['descriptions']:
                    event_data['descriptions'].add(execution_desc)
                    event_data['types'].add('execution')
                
                if state_frag_desc and state_frag_desc not in event_data['descriptions']:
                    event_data['descriptions'].add(state_frag_desc)
                    event_data['types'].add('state')

# Helper function to extract timepoint order number
def get_tp_order(tp_name):
    match = re.search(r'(\d+)$', tp_name)
    return int(match.group(1)) if match else 0

# Helper function to truncate long lifeline names
def truncate_lifeline(name, max_len=20):
    if len(name) <= max_len:
        return name
    return name[:max_len-3] + '...'

# Store figures for HTML export
scenarios_figs = []

# Create visualizations for each scenario
for scenario_name, scenario_info in scenarios_data.items():
    if not scenario_info['lifelines']:
        continue
        
    print(f"\n{'='*80}")
    print(f"Scenario: {scenario_name}")
    print(f"Description: {scenario_info['description']}")
    print(f"{'='*80}\n")
    
    # Sort lifelines by name for consistent display
    lifeline_names = sorted(scenario_info['lifelines'].keys())
    
    # Sort timepoints by the number at the end
    timepoint_names = sorted(scenario_info['timepoints'].keys(), key=get_tp_order)
    
    # Create figure
    fig = go.Figure()
    
    num_timepoints = len(timepoint_names)
    num_lifelines = len(lifeline_names)
    
    # Add lifeline columns with headers
    for i, lifeline_name in enumerate(lifeline_names):
        x_pos = i
        lifeline_desc = scenario_info['lifelines'][lifeline_name]
        truncated_name = truncate_lifeline(lifeline_name)
        
        # Draw vertical lifeline
        fig.add_trace(go.Scatter(
            x=[x_pos, x_pos],
            y=[0, num_timepoints + 0.5],
            mode='lines',
            line=dict(color='lightgray', width=2, dash='dot'),
            showlegend=False,
            hoverinfo='skip'
        ))
        
        # Add lifeline header with hover showing full name and description
        fig.add_trace(go.Scatter(
            x=[x_pos],
            y=[num_timepoints + 1],
            mode='text',
            text=[f"<b>{truncated_name}</b>"],
            textposition='top center',
            textfont=dict(size=12, color='darkblue'),
            hovertext=f"<b>{lifeline_name}</b><br>{lifeline_desc}",
            hoverinfo='text',
            showlegend=False
        ))
    
    # Add events at each timepoint
    for row_idx, tp_name in enumerate(timepoint_names):
        y_pos = num_timepoints - row_idx  # Top to bottom
        
        # Add timepoint label on the left
        fig.add_annotation(
            x=-0.5,
            y=y_pos,
            text=f"<b>{tp_name}</b>",
            showarrow=False,
            font=dict(size=10, color='darkgreen'),
            xanchor='right',
            yanchor='middle'
        )
        
        # Check each lifeline for events at this timepoint
        for col_idx, lifeline_name in enumerate(lifeline_names):
            x_pos = col_idx
            
            event_data = scenario_info['timepoints'][tp_name].get(lifeline_name, {})
            descriptions = event_data.get('descriptions', set())
            types = event_data.get('types', set())
            
            if descriptions:
                # Build combined text with appropriate emoji prefix based on what types are present
                event_parts = []
                
                # Determine the primary type for emoji selection
                if 'message' in types:
                    emoji = '📧'
                elif 'execution' in types:
                    emoji = '⚙️'
                elif 'state' in types:
                    emoji = '📍'
                else:
                    emoji = '•'
                
                # Add all unique descriptions with the chosen emoji
                for desc in sorted(descriptions):
                    event_parts.append(f"{emoji} {desc}")
                
                combined_text = '<br>'.join(event_parts)
                
                # Event marker
                fig.add_trace(go.Scatter(
                    x=[x_pos],
                    y=[y_pos],
                    mode='markers',
                    marker=dict(size=15, color='steelblue', symbol='circle', line=dict(width=2, color='darkblue')),
                    hovertext=f"<b>{tp_name}</b> on <b>{lifeline_name}</b><br>{combined_text}",
                    hoverinfo='text',
                    showlegend=False
                ))
    
    # Update layout
    fig.update_layout(
        title=f"<b>{scenario_name}</b><br><sub>{scenario_info['description']}</sub>",
        xaxis=dict(
            showgrid=False,
            showticklabels=False,
            zeroline=False,
            range=[-1, num_lifelines]
        ),
        yaxis=dict(
            showgrid=False,
            showticklabels=False,
            zeroline=False,
            range=[-0.5, num_timepoints + 1.5]
        ),
        plot_bgcolor='white',
        height=300 + (num_timepoints * 50),
        hovermode='closest',
        margin=dict(l=100, r=20, t=100, b=20)
    )
    
    fig.show()
    
    # Store for HTML export
    scenarios_figs.append((scenario_name, fig))

print(f"\n✅ Visualized {len(scenarios_data)} scenarios")



Scenario: ApproveAIWardenTask
Description: Scenario for the human-in-the-loop approval of an AI Warden task proposal.




Scenario: RespondToCriticalNotification
Description: Scenario showing flow when a critical notification is produced and acknowledged.




Scenario: ViewHistoricalData
Description: Scenario demonstrating an Operator viewing historical fire states on the dashboard.




✅ Visualized 3 scenarios


In [109]:
# ==========================================
# Cell 7: Export all visuals into one HTML dashboard
# ==========================================
from pathlib import Path
import json as json_module

table_html = df.to_html(classes='table table-striped table-bordered', index=False)
fig_missions_html = fig_missions.to_html(full_html=False, include_plotlyjs=False)
fig_entities_html = fig_entities.to_html(full_html=False, include_plotlyjs=False)
fig_activities_html = fig_activities.to_html(full_html=False, include_plotlyjs=False)

# Generate HTML for all scenarios
scenarios_html_sections = []
for scenario_name, scenario_fig in scenarios_figs:
    scenario_html = scenario_fig.to_html(full_html=False, include_plotlyjs=False)
    scenarios_html_sections.append(f"""
    <h1 style="margin-top:3rem;">{scenario_name} Timeline</h1>
    {scenario_html}
    """)
scenarios_combined_html = '\n'.join(scenarios_html_sections)

# Create Mermaid HTML with tooltip data (properly escaped for HTML)
tooltip_data = {
    'nodes': {name: desc for name, desc in statemachine_nodes.items()},
    'edges': {f"{s1}->{s2}": {'trigger': tn or '', 'description': td or ''} 
              for (s1, s2), (tn, td) in statemachine_edges.items()}
}
tooltip_json = json_module.dumps(tooltip_data)

fig_statemachine_html = f'''<div id="statemachine-container" style="position: relative;">
    <div class="mermaid" id="statemachine-diagram">{html_module.escape(fig_statemachine_mermaid)}</div>
    <div id="sm-tooltip" style="position: absolute; display: none; background: rgba(0,0,0,0.85); color: white; padding: 8px 12px; border-radius: 4px; font-size: 12px; pointer-events: none; z-index: 1000; max-width: 300px; box-shadow: 0 2px 8px rgba(0,0,0,0.3);"></div>
</div>
<script>
const smTooltipData = {tooltip_json};
</script>'''

html = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>Mission Control Dashboard</title>
    <link rel="stylesheet"
          href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
    <link rel="stylesheet"
          href="https://cdn.datatables.net/1.13.6/css/dataTables.bootstrap5.min.css">
    <script src="https://cdn.jsdelivr.net/npm/jquery@3.7.1/dist/jquery.min.js"></script>
    <script src="https://cdn.datatables.net/1.13.6/js/jquery.dataTables.min.js"></script>
    <script src="https://cdn.datatables.net/1.13.6/js/dataTables.bootstrap5.min.js"></script>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <script type="module">
      import mermaid from 'https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs';
      mermaid.initialize({{ startOnLoad: true }});
      
      // Store reference for tooltip attachment after Mermaid renders
      window.mermaidReady = new Promise((resolve) => {{
        window.addEventListener('load', () => {{
          setTimeout(resolve, 2000);
        }});
      }});
    </script>
    <style>
      body {{ margin: 2rem; }}
      h1 {{ margin-bottom: 1.5rem; }}
    </style>
</head>
<body>
    <h1>Requirements Table</h1>
    <div class="table-responsive">
      {table_html}
    </div>

    <h1 style="margin-top:3rem;">Requirement–Capability–SubCapability Graph</h1>
    {fig_missions_html}

    <h1 style="margin-top:3rem;">Capability–Entity Graph</h1>
    {fig_entities_html}

    <h1 style="margin-top:3rem;">Activity–Data Flow Graph</h1>
    {fig_activities_html}

    <h1 style="margin-top:3rem;">State Machine Transition Graph</h1>
    {fig_statemachine_html}

    {scenarios_combined_html}

    <script>
    $(document).ready(function() {{
        $('table').DataTable({{
            paging: true,
            searching: true,
            ordering: true,
            pageLength: 10
        }});
        
        // Wait for Mermaid to fully render before attaching tooltips
        async function attachTooltips() {{
            if (typeof window.mermaidReady !== 'undefined') {{
                await window.mermaidReady;
            }}
            
            // Additional wait to ensure Mermaid SVG is in DOM
            await new Promise(resolve => setTimeout(resolve, 500));
            
            const container = document.getElementById('statemachine-container');
            const tooltip = document.getElementById('sm-tooltip');
            
            if (!container || !tooltip || typeof smTooltipData === 'undefined') {{
                console.error('Tooltip setup failed - missing elements');
                return;
            }}
            
            // Helper to show tooltip
            function showTooltip(content, e) {{
                tooltip.innerHTML = content;
                tooltip.style.display = 'block';
                updateTooltipPosition(e);
            }}
            
            function hideTooltip() {{
                tooltip.style.display = 'none';
            }}
            
            function updateTooltipPosition(e) {{
                const containerRect = container.getBoundingClientRect();
                tooltip.style.left = (e.clientX - containerRect.left + 15) + 'px';
                tooltip.style.top = (e.clientY - containerRect.top + 15) + 'px';
            }}
            
            // Add tooltips to state nodes
            const states = container.querySelectorAll('g.node');
            console.log('Found ' + states.length + ' state nodes');
            
            states.forEach(state => {{
                // Get the data-id attribute which contains the state name
                const stateName = state.getAttribute('data-id');
                if (!stateName) return;
                
                console.log('Processing state: ' + stateName);
                
                if (smTooltipData.nodes[stateName]) {{
                    // Find the rect and text elements to attach events to
                    const rect = state.querySelector('rect.basic');
                    const textElements = state.querySelectorAll('text, foreignObject');
                    
                    if (!rect) return;
                    
                    const tooltipContent = `<strong>${{stateName}}</strong><br>${{smTooltipData.nodes[stateName]}}`;
                    
                    // Attach to rect
                    rect.style.cursor = 'pointer';
                    rect.style.pointerEvents = 'all';
                    rect.addEventListener('mouseenter', function(e) {{
                        console.log('Hovering over state: ' + stateName);
                        showTooltip(tooltipContent, e);
                    }});
                    rect.addEventListener('mousemove', updateTooltipPosition);
                    rect.addEventListener('mouseleave', hideTooltip);
                    
                    // Also attach to text elements
                    textElements.forEach(textEl => {{
                        textEl.style.cursor = 'pointer';
                        textEl.style.pointerEvents = 'all';
                        textEl.addEventListener('mouseenter', function(e) {{
                            showTooltip(tooltipContent, e);
                        }});
                        textEl.addEventListener('mousemove', updateTooltipPosition);
                        textEl.addEventListener('mouseleave', hideTooltip);
                    }});
                }}
            }});
            
            // Add tooltips to transitions (edge labels)
            const edgeLabels = container.querySelectorAll('g.edgeLabel');
            console.log('Found ' + edgeLabels.length + ' edge labels');
            
            edgeLabels.forEach(edgeLabel => {{
                const span = edgeLabel.querySelector('.edgeLabel');
                if (!span) return;
                
                const triggerName = span.textContent.trim();
                if (!triggerName) return;
                
                console.log('Processing edge label: ' + triggerName);
                
                // Find matching edge in tooltipData
                for (const [edgeKey, edgeData] of Object.entries(smTooltipData.edges)) {{
                    if (edgeData.trigger === triggerName && edgeData.description) {{
                        const rect = edgeLabel.querySelector('rect');
                        const textElements = edgeLabel.querySelectorAll('text, foreignObject');
                        if (!rect) continue;
                        
                        const tooltipContent = `<strong>${{triggerName}}</strong><br>${{edgeData.description}}`;
                        
                        // Attach to rect
                        rect.style.cursor = 'pointer';
                        rect.style.pointerEvents = 'all';
                        rect.addEventListener('mouseenter', function(e) {{
                            console.log('Hovering over transition: ' + triggerName);
                            showTooltip(tooltipContent, e);
                        }});
                        rect.addEventListener('mousemove', updateTooltipPosition);
                        rect.addEventListener('mouseleave', hideTooltip);
                        
                        // Also attach to text elements
                        textElements.forEach(textEl => {{
                            textEl.style.cursor = 'pointer';
                            textEl.style.pointerEvents = 'all';
                            textEl.addEventListener('mouseenter', function(e) {{
                                showTooltip(tooltipContent, e);
                            }});
                            textEl.addEventListener('mousemove', updateTooltipPosition);
                            textEl.addEventListener('mouseleave', hideTooltip);
                        }});
                        break;
                    }}
                }}
            }});
            
            console.log('Tooltip attachment complete');
        }}
        
        attachTooltips();
    }});
    </script>
</body>
</html>
"""

out_path = Path("mission_dashboard.html")
out_path.write_text(html, encoding="utf-8")
print(f"✅ Dashboard saved to {out_path.resolve()}")


✅ Dashboard saved to /Users/vivek/Notes/CS_188/mission-control/notebooks/mission_dashboard.html
